In [1]:
# Using environment Qiskit 1.2.0
!pip install scikit-learn

In [2]:
# Importing the required libraries (install in your environment first)
import numpy as np
import pandas as pd
from math import log2, sqrt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_svmlight_file
from scipy.stats import chi2, binom
from scipy.fft import fft
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report, accuracy_score
import xgboost as xgb
from sklearn import svm



In [3]:
# Read in data from your datafile or the provided datafile in this folder to classify QRNG data. 
# Suggested classification strategies include QRNG vs PRNG, QPU vs Simulator, or by individual QPU

#Sample datafile of QRNG (IBM QPUs) vs PRNG data, 12k lines each. Label 1 is QRNG, label 2 is PRNG

# Read data file, make dataframe, process labels, and combine/concatenate individual input lines into
# larger input lines to create training and testing datasets to input into gradient boosting model.
# Hint: use train_test_split method from sklearn

def read_file_in_chunks(filename, N):
    with open(filename, 'r') as file:
        data = file.read().strip()  # Read the entire file into a single string and remove any trailing spaces or newlines
    return [data[i:i+N] for i in range(0, len(data), N)]  # Split the string into chunks of size N

# Read from RNG1 and RNG2
N = 1000
rng1_bitstrings = read_file_in_chunks('RNG1-2.txt', N)
rng2_bitstrings = read_file_in_chunks('RNG2-2.txt', N)

# Create DataFrames for RNG1 and RNG2, and assign labels
df_rng1 = pd.DataFrame({
    'bitstrings': rng1_bitstrings,
    'label': 1  # Label all RNG1 strings as '1'
})

df_rng2 = pd.DataFrame({
    'bitstrings': rng2_bitstrings,
    'label': 0  # Label all RNG2 strings as '2'
})

# Combine both DataFrames
df = pd.concat([df_rng1, df_rng2], ignore_index=True)

In [4]:
# Example DataFrame (assuming you already have it)
# df['bitstrings'] contains the bitstrings of length 100
# df = pd.DataFrame({'bitstrings': ['1100100100001111110110...', '1010101010101010101010...', ...]})

# Function to calculate Shannon entropy
def shannon_entropy(bitstring):
    # Convert bitstring to a numpy array of integers
    bits = np.array([int(bit) for bit in bitstring])
    
    # Calculate the frequency of 0's and 1's
    counts = np.bincount(bits)
    probabilities = counts / len(bits)
    
    # Filter out zero probabilities to avoid log2(0)
    probabilities = probabilities[probabilities > 0]
    
    # Calculate entropy
    entropy = -np.sum(probabilities * np.log2(probabilities))
    
    return entropy

# Apply the entropy function to each bitstring in the DataFrame
df['entropy'] = df['bitstrings'].apply(shannon_entropy)


In [5]:
def chi_squared_test(bits):
        counts = np.bincount(list(bits), minlength=2)
        observed = counts
        expected = np.array([len(bits)/2, len(bits)/2])
        chi_sq = np.sum((observed - expected) ** 2 / expected)
        # Degrees of freedom = number of categories - 1 = 1
        p_value = 1 - chi2.cdf(chi_sq, df=1)
        return chi_sq, p_value

# Apply the function and store results in two new columns
df[['chi_sq', 'chi_sq_p_value']] = pd.DataFrame(df['bitstrings'].apply(chi_squared_test).tolist(), index=df.index)


In [6]:

# Function to get the top 5 dominant magnitudes of frequencies
def dominant_frequencies(bitstring, top_n=5):
    # Convert bitstring to list of integers (0s and 1s)
    bits = np.array([int(bit) for bit in bitstring])
    n = len(bits)  # Length of the bitstring

    # Convert bits to -1 and 1 for FFT
    signal = 2 * bits - 1
    fft_result = fft(signal)
    
    # Only take the positive frequencies
    freqs = np.fft.fftfreq(n)
    magnitudes = np.abs(fft_result)
    positive_freqs = freqs[:n//2]
    positive_magnitudes = magnitudes[:n//2]
    
    # Find the top_n dominant frequencies
    indices = np.argsort(positive_magnitudes)[-top_n:]
    dominant_mags = positive_magnitudes[indices]

    # Sort the dominant magnitudes in descending order
    sorted_mags = np.sort(dominant_mags)[::-1]
    
    # If fewer than top_n magnitudes, pad with NaNs
    if len(sorted_mags) < top_n:
        sorted_mags = np.pad(sorted_mags, (0, top_n - len(sorted_mags)), constant_values=np.nan)
    
    return sorted_mags

# Apply the function and expand it into separate columns
df[['freq_mag_1', 'freq_mag_2', 'freq_mag_3', 'freq_mag_4', 'freq_mag_5']] = pd.DataFrame(
    df['bitstrings'].apply(dominant_frequencies).tolist(), index=df.index
)


In [7]:
def autocorrelation(bitstring, lag=1):
    # Convert the bitstring to a numpy array of integers (0 and 1)
    bits = np.array([int(bit) for bit in bitstring])
    
    if lag >= len(bits):
        raise ValueError("Lag is too large for the bitstream length.")
    
    # Shift the bits by the given lag
    shifted = np.roll(bits, -lag)
    
    # Calculate correlation excluding the wrapped-around elements
    valid_length = len(bits) - lag
    correlation = np.corrcoef(bits[:valid_length], shifted[:valid_length])[0, 1]
    
    return correlation

df['autocorrelation_lag1'] = df['bitstrings'].apply(autocorrelation)

In [8]:
def frequency_test(bitstring):
    """
    Perform the Frequency Test on a bitstring.
    
    :param bitstring: A 1D numpy array or list of bits (0s and 1s).
    :return: A tuple containing the number of 1s, number of 0s, and the test statistic.
    """
    count_ones = list(bitstring).count('1')
    count_zeros = len(bitstring) - count_ones
    
    return count_ones, count_zeros

def block_frequency_test(bitstring, M):
    """
    Perform the Block Frequency Test on a bitstring.
    
    :param bitstring: A 1D numpy array or list of bits (0s and 1s).
    :param M: Block size for the block frequency test.
    :return: The frequencies of 1s in each block and the test statistic.
    """
    bitstring = [int(c) for c in bitstring]
    n = len(bitstring)
    num_blocks = n // M
    block_frequencies = np.zeros(num_blocks)

    # Compute block frequencies
    for i in range(num_blocks):
        block = bitstring[i * M:(i + 1) * M]
        block_frequencies[i] = np.sum(block)

    # Calculate the mean and variance of the block frequencies
    mean_frequency = np.mean(block_frequencies)
    variance_frequency = np.var(block_frequencies)

    # The expected mean and variance for a random sequence
    expected_mean = M / 2
    expected_variance = M / 4

    # Chi-squared statistic
    chi_squared = ((mean_frequency - expected_mean) ** 2 / expected_variance) + \
                  (variance_frequency / expected_variance)

    return block_frequencies.max(), block_frequencies.mean(), chi_squared

df['freq_ones'], df['freq_zeros'] = zip(*df['bitstrings'].apply(frequency_test))

# Set block size for Block Frequency Test
M = 10  # Example block size
df['block_freqs_max'], df['block_freqs_mean'], df['block_freq_chi_squared'] = zip(*df['bitstrings'].apply(lambda x: block_frequency_test(x, M)))


In [9]:

def runs_test(bits):
    """
    Performs the Runs Test on a binary sequence.
    
    Parameters:
    bits (array-like): A binary sequence (1s and 0s).
    
    Returns:
    (int, float): The number of runs and the p-value of the test.
    """
    # Convert bits to a numpy array
    bits = np.array([int(c) for c in bits])
    
    # Count the number of runs
    runs = 1  # Start with the first run
    for i in range(1, len(bits)):
        if bits[i] != bits[i-1]:
            runs += 1
            
    n1 = np.sum(bits)  # Number of 1s
    n0 = len(bits) - n1  # Number of 0s
    
    # Calculate the expected number of runs and variance
    expected_runs = (2 * n1 * n0) / (n1 + n0) + 1
    variance_runs = (2 * n1 * n0 * (2 * n1 * n0 - n1 - n0)) / ((n1 + n0) ** 2 * (n1 + n0 - 1))
    
    # Z-score
    z = (runs - expected_runs) / np.sqrt(variance_runs)
    
    # Calculate p-value from z-score
    p_value = 1 - binom.cdf(runs, n=n1 + n0, p=0.5)  # Note: binom.cdf is not directly usable for z-scores
    
    return runs, p_value

df['runs'], df['runs_p_value'] = zip(*df['bitstrings'].apply(runs_test))

In [10]:
def xor(a, b):
    """
    Returns the XOR of two binary values (0 or 1).
    
    Parameters:
    a (int): First binary value (0 or 1).
    b (int): Second binary value (0 or 1).
    
    Returns:
    int: Result of a XOR b (0 or 1).
    """
    return (a or b) and not (a and b)

def linear_complexity(bits):
    """
    Computes the Linear Complexity of a binary sequence using the Berlekamp-Massey algorithm.
    
    Parameters:
    bits (array-like): A binary sequence (1s and 0s).
    
    Returns:
    int: The linear complexity of the sequence.
    """
    bits = np.array([int(c) for c in bits])
    n = len(bits)
    l = 0  # Linear complexity
    m = -1  # Previous index where the error occurred
    C = np.zeros(n)
    B = np.zeros(n)
    C[0] = 1  # The polynomial is initialized with a leading coefficient of 1
    B[0] = 1
    
    for n in range(n):
        # Calculate the discrepancy
        discrepancy = bits[n]
        for i in range(1, l + 1):
            discrepancy = xor(discrepancy, (C[i] * bits[n - i]))
        
        if discrepancy == 1:  # An error occurred
            T = C.copy()
            for i in range(n - m):
                if m + i < n:
                    C[m + i] = xor(C[m + i], B[i])
            if l <= n // 2:
                l = n + 1 - l
                m = n
                B = T.copy()
    
    return l

df['linear_complexity'] = df['bitstrings'].apply(linear_complexity)

In [11]:
# Normalize a specific column in the DataFrame (let's assume 'column_name' is the one you want to normalize)
def zscore_col(df, colname):
    df[colname] = (df[colname] - df[colname].mean()) / df[colname].std()

#metrics = ['linear_complexity', 'runs_p_value', 'runs', 'chi_sq','chi_sq_p_value','entropy', 'freq_ones', 'freq_zeros', 'block_freqs_max', 'block_freqs_mean', 'block_freq_chi_squared']
metrics = ['freq_mag_1', 'freq_mag_2', 'freq_mag_3', 'freq_mag_4', 'freq_mag_5', 'autocorrelation_lag1', 'linear_complexity', 'runs_p_value', 'runs', 'chi_sq','chi_sq_p_value','entropy', 'freq_ones', 'freq_zeros', 'block_freqs_max', 'block_freqs_mean', 'block_freq_chi_squared']



for colname in metrics:
    zscore_col(df, colname)

# Calculate correlation
correlation_matrix = df[metrics + ['label']].corr()
correlation_with_target = correlation_matrix['label'].sort_values(ascending=False, key=abs)
print(correlation_with_target)


label                     1.000000
chi_sq                    0.195938
entropy                  -0.195914
chi_sq_p_value           -0.152670
block_freqs_mean          0.105297
freq_zeros               -0.105297
freq_ones                 0.105297
block_freq_chi_squared    0.067186
freq_mag_4                0.020589
block_freqs_max           0.017653
freq_mag_5                0.017355
runs_p_value              0.017164
runs                     -0.014431
freq_mag_2                0.012462
freq_mag_3                0.012420
freq_mag_1                0.010301
linear_complexity        -0.003191
autocorrelation_lag1      0.002393
Name: label, dtype: float64


In [12]:
# Split the 'strings' column into separate columns for each character
df_split = df['bitstrings'].str.split('', expand=True)

# Fill NaN values with an empty string (optional)
df_split.fillna('', inplace=True)

# Concatenate the split DataFrame with the original DataFrame
df_combined = pd.concat([df, df_split], axis=1)

In [13]:
# Prepare input features (X) and target labels (y)
X = pd.DataFrame(df[metrics])

X = np.array(X)
y = np.array(df['label'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

[[ 0.11331242  0.70308799  1.38687271 ...  0.3937202  -0.15626922
  -2.76190975]
 [ 2.14834186  1.5427716  -0.15725071 ...  0.3937202   2.16616504
  -0.82426788]
 [-0.18943043 -0.67502495 -0.40451501 ... -1.23546684 -0.53433991
  -2.07628263]
 ...
 [-1.34594285 -0.91642668 -0.38062398 ...  0.3937202  -1.99261259
  -0.85407776]
 [-1.16045472 -0.56543332 -0.95388493 ...  0.3937202  -1.2364712
  -1.15217651]
 [-0.91257528 -0.51149325 -0.9529761  ...  0.3937202  -1.07444091
   0.90470486]]
[0 1 1 ... 1 0 1]


In [14]:
def tune_gradient_boosting(X_train, y_train, n_iter=100, random_state=42):
    # Define the parameter grid
    param_dist = {
        'n_estimators': np.arange(50, 300, 50),  # Number of boosting stages
        'learning_rate': [0.001, 0.01, 0.1, 0.2, 0.3],  # Learning rate
        'max_depth': np.arange(3, 11),  # Maximum depth of the individual regression estimators
        'min_samples_split': np.arange(2, 11),  # Minimum number of samples required to split an internal node
        'min_samples_leaf': np.arange(1, 11),  # Minimum number of samples required to be at a leaf node
        'subsample': [0.5, 0.7, 1.0],  # Fraction of samples to be used for fitting the individual base learners
        'max_features': [None, 'sqrt', 'log2']  # Number of features to consider when looking for the best split
    }

    # Create the Gradient Boosting Classifier
    gb_classifier = GradientBoostingClassifier(random_state=random_state)

    # Setup RandomizedSearchCV
    random_search = RandomizedSearchCV(
        estimator=gb_classifier,
        param_distributions=param_dist,
        n_iter=n_iter,
        scoring='accuracy',
        cv=5,  # 5-fold cross-validation
        verbose=1,
        random_state=random_state,
        n_jobs=-1  # Use all available cores
    )

    # Fit the model
    random_search.fit(X_train, y_train)

    # Get the best parameters and model
    best_params = random_search.best_params_
    best_model = random_search.best_estimator_

    return best_params, best_model

# Tune the Gradient Boosting model
best_params, best_model = tune_gradient_boosting(X_train, y_train)

    # Output the best parameters
print("Best Hyperparameters:", best_params)

    # Make predictions with the best model
y_pred = best_model.predict(X_train)
print("Accuracy:", accuracy_score(y_train, y_pred))
print(classification_report(y_train, y_pred))

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Best Hyperparameters: {'subsample': 1.0, 'n_estimators': 100, 'min_samples_split': 9, 'min_samples_leaf': 9, 'max_features': 'log2', 'max_depth': 3, 'learning_rate': 0.001}
Accuracy: 0.5877777777777777
              precision    recall  f1-score   support

           0       0.56      0.89      0.68       903
           1       0.72      0.28      0.41       897

    accuracy                           0.59      1800
   macro avg       0.64      0.59      0.54      1800
weighted avg       0.64      0.59      0.55      1800



In [15]:
# A skeleton for running your training dataframe through a SKLearn gradient boosting model. 
# You can also use other ML frameworks such as Pytorch, XGBoost, etc
# Create the Gradient Boosting classifier
xgb_model = xgb.XGBClassifier(**best_params)

# Train the model. Define X_train and y_train from your training dataframe using 
# sklearns train_test_split method
xgb_model.fit(X_train, y_train, eval_set=[(X_test, y_test)])

y1 = xgb_model.predict(X_train)
accuracy = accuracy_score(y_train, y1)
print("Training accuracy: ", accuracy)

# Make predictions on the test set
y_pred_gb = xgb_model.predict(X_test)

# Calculate the accuracy of the Gradient Boosting model
accuracy_gb = accuracy_score(y_test, y_pred_gb)
print("Gradient Boosting Accuracy for XGB: ", accuracy_gb)

[0]	validation_0-logloss:0.69314
[1]	validation_0-logloss:0.69310
[2]	validation_0-logloss:0.69306
[3]	validation_0-logloss:0.69302
[4]	validation_0-logloss:0.69298
[5]	validation_0-logloss:0.69294
[6]	validation_0-logloss:0.69290
[7]	validation_0-logloss:0.69286
[8]	validation_0-logloss:0.69281
[9]	validation_0-logloss:0.69277
[10]	validation_0-logloss:0.69272
[11]	validation_0-logloss:0.69268
[12]	validation_0-logloss:0.69264
[13]	validation_0-logloss:0.69260
[14]	validation_0-logloss:0.69256
[15]	validation_0-logloss:0.69252
[16]	validation_0-logloss:0.69247
[17]	validation_0-logloss:0.69243
[18]	validation_0-logloss:0.69239
[19]	validation_0-logloss:0.69235
[20]	validation_0-logloss:0.69231
[21]	validation_0-logloss:0.69227
[22]	validation_0-logloss:0.69222
[23]	validation_0-logloss:0.69218
[24]	validation_0-logloss:0.69214
[25]	validation_0-logloss:0.69210
[26]	validation_0-logloss:0.69206
[27]	validation_0-logloss:0.69202
[28]	validation_0-logloss:0.69198
[29]	validation_0-loglos

/home/jovyan/.qbraid/environments/qiskit_9vrlwn/pyenv/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [23:15:58] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "max_features", "min_samples_leaf", "min_samples_split" } are not used.

  warnings.warn(smsg, UserWarning)


In [16]:
clf = svm.SVC(kernel='rbf', C=0.7)
clf.fit(X_train, y_train)

y1 = clf.predict(X_train)
accuracy = accuracy_score(y_train, y1)
#print("Training accuracy: ", accuracy)

# Make predictions on the test set
y_pred_gb = clf.predict(X_test)

# Calculate the accuracy of the Gradient Boosting model
accuracy_gb = accuracy_score(y_test, y_pred_gb)
print(f"Testing Accuracy For SVM: ", accuracy_gb)

Testing Accuracy For SVM:  0.6016666666666667


In [17]:
# A skeleton for running your training dataframe through a SKLearn gradient boosting model. 
# You can also use other ML frameworks such as Pytorch, XGBoost, etc

# Create the Gradient Boosting classifier
#gb_model = GradientBoostingClassifier(random_state=42, subsample=0.8, n_estimators=100, max_depth=11, loss='log_loss', min_samples_leaf=2, learning_rate=0.001)
gb_model = GradientBoostingClassifier(
    **best_params
)
# Train the model. Define X_train and y_train from your training dataframe using 
# sklearns train_test_split method
gb_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_gb = gb_model.predict(X_test)

y1 = gb_model.predict(X_train)
accuracy = accuracy_score(y_train, y1)
print("Training accuracy: ", accuracy)

# Calculate the accuracy of the Gradient Boosting model
accuracy_gb = accuracy_score(y_test, y_pred_gb)
print("Gradient Boosting Accuracy for GB: ", accuracy_gb)

Training accuracy:  0.595
Gradient Boosting Accuracy for GB:  0.5683333333333334
